In [2]:
import os

import numpy as np
import xarray as xr

from pyrte_rrtmgp import rrtmgp_gas_optics
from pyrte_rrtmgp.rrtmgp_gas_optics import GasOpticsFiles, load_gas_optics
from pyrte_rrtmgp.rrtmgp_data import download_rrtmgp_data
from pyrte_rrtmgp.rte_solver import rte_solve

ERROR_TOLERANCE = 1e-7

rte_rrtmgp_dir = download_rrtmgp_data()
rfmip_dir = os.path.join(rte_rrtmgp_dir, "examples", "rfmip-clear-sky")
input_dir = os.path.join(rfmip_dir, "inputs")
ref_dir = os.path.join(rfmip_dir, "reference")

gas_optics_sw = load_gas_optics(gas_optics_file=GasOpticsFiles.SW_G224)

atmosphere_file = "multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc"
atmosphere_path = os.path.join(input_dir, atmosphere_file)
atmosphere = xr.load_dataset(atmosphere_path).sel(expt=0)  # only one experiment

gas_optics_sw.gas_optics.compute(atmosphere, problem_type="two-stream")

fluxes = rte_solve(atmosphere, add_to_input=False)

rsu_reference = f"{ref_dir}/rsu_Efx_RTE-RRTMGP-181204_rad-irf_r1i1p1f1_gn.nc"
rsd_reference = f"{ref_dir}/rsd_Efx_RTE-RRTMGP-181204_rad-irf_r1i1p1f1_gn.nc"

# Compare the results with the reference data
rsu = xr.load_dataset(rsu_reference, decode_cf=False)
ref_flux_up = rsu.isel(expt=0)["rsu"].values

rsd = xr.load_dataset(rsd_reference, decode_cf=False)
ref_flux_down = rsd.isel(expt=0)["rsd"].values

assert np.isclose(fluxes["sw_flux_up"].values, ref_flux_up, atol=ERROR_TOLERANCE).all()
assert np.isclose(fluxes["sw_flux_down"].values, ref_flux_down, atol=ERROR_TOLERANCE).all()